<a href="https://colab.research.google.com/github/chene/ARQOPUS/blob/main/line_Procrustes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Line-based Orthogonal Procrustes Analysis

In line-based Procrustes Analysis, we are interested in register a set of points to a set of lines, where the correspondence between point-line is explicitly known. Within the context of computer-assisted interventions (CAI), we are typically interested in **scaling**, followed by **rotation** and **translation** of a set of 3D *points* so they are registered (aligned) to a set of 3D *lines*. Again, Orthogonal implies that there is no searing. We also assume there is a 1:1 correspondence between these two 3D datasets.

The immediate application for line-based Procrustes Analysis in CAI are:

Camera Hand-Eye Calibration:
```
@article{Morgan2017,
author    = {Morgan, Isabella and Jayarathne, Uditha L. and Rankin, Adam and Peters, Terry M. and Chen, Elvis C. S.},
doi       = {10.1007/s11548-017-1590-9},
journal   = {International journal of computer assisted radiology and surgery},
number    = {7},
pages     = {1141--1149},
pmid      = {28425030},
title     = {Hand-eye calibration for surgical cameras: a Procrustean Perspective-n-Point solution},
volume    = {12},
year      = {2017}, }
```

and Ultrasound Probe Calibration:
```
@article{Chen2016,
author    = {Chen, Elvis C. S. and Peters, Terry M. and Ma, Burton},
doi       = {10.1007/s11548-016-1390-7},
journal   = {International Journal of Computer Assisted Radiology and Surgery},
number    = {6},
pages     = {889--898},
pmid      = {27038966},
title     = {Guided ultrasound calibration: where, how, and how many calibration fiducials},
volume    = {11},
year      = {2016}, }
```

I first introduced the Matlab code in the following:
```
@inproceedings{Chen2017,
author    = {Chen, Elvis C. S. and Peters, Terry M. and Ma, Burton},
booktitle = {Proc. SPIE 10135, Medical Imaging 2017: Image-Guided Procedures, Robotic Interventions, and Modeling},
doi       = {10.1117/12.2253692},
editor    = {Webster, Robert J. and Fei, Baowei},
pages     = {1013509},
title     = {Which point-line registration?},
volume    = {1013509},
year      = {2017}, }
```

We address the problem of line-based Procrustean Analysis using an iterative algorithm. It works in a similar manner to the popular [ICP](https://ieeexplore.ieee.org/document/121791) algorithm, where the corresponding point for **a point** along **a given line** is searched first, followed by [the point-based Procrustean Analysis](https://github.com/chene/ARQOPUS/blob/main/point_Procrustes.ipynb). 

The Matlab code is:
```Matlab
function [ R, t, A ] = p2l( X, Y, D, tol )
%
% INPUT:
%        X : matrix (3xn) 3D coordinates
%        Y : matrix (3xn) 3D coordinates, line origin
%        D : matrix (3xn) 3D vector, line orientation
%        tol: exit condition
%
% OUTPUT:
%         solves the registration between points (X) and line (Y+tD)
%
%        Q : 3x3 rotation
%        t : 3x1 translation
%        A : 3x3 diagonal scales
%
% Elvis C.S. Chen
% chene AT robarts DOT ca

[m,n] = size(X); e = ones(1,n); err = +Inf;
E_old = 1000*ones( m,n ); Q = Y; Dir = normc(D);
while err > tol
    [ R, t, A ] = AOPA_Major( X, Q, tol);                % pont-based solution
    E = Q - R*A*X-t*e;                                   % correspondence
    Q = Y + Dir .* repmat( dot(R*A*X+t*e-Y,Dir),[m,1] ); % reprojection
    err = norm(E-E_old,'fro'); E_old = E;                % residual error
end
```
where a line is represented as a line origin ($Y$, 3D points) and its unit orientation ($D$).

Alternatively, the line parameterization $t$ can be solved independently, which provides point correspondence:
```Matlab
function [R, t, A] = g2Fiore(X, O, D, tol)
%
% INPUT:
%        X : matrix (3xn) 3D coordinates
%        Y : matrix (3xn) 3D coordinates, line origin
%        D : matrix (3xn) 3D vector, line orientation
%        tol: exit condition
%
% OUTPUT:
%         solves the registration between points (X) and line (Y+tD)
%
%        Q : 3x3 rotation
%        t : 3x1 translation
%        A : 3x3 diagonal scales
%
% Elvis C.S. Chen
% chene AT robarts DOT ca
n = size(D, 2); e = ones( 1, n );
M = [X; e]; [~,~,V] = svd(M); V2 = V(:, rank(M)+1:end);
D = ( (D'*D) .*(V2*V2') ); b = -diag( V2*V2'*O'*D );
v = S\b; Z = diag(v);
[R, t, A] = AOPA_major(X, D*Z+O, tol);
```
refer to my [SPIE paper](https://www.spiedigitallibrary.org/conference-proceedings-of-spie/10135/1013509/Which-point-line-registration/10.1117/12.2253692.full?SSO=1) for derivation.  For both of these solutions, a minimum of $6$ paired point-line are required. Note that when the cardinality of the dataset is large, the $g2Fiore$ algorithm may be computationally expensive and at times unstable due to the use of matrix left-division.

In my experience, the $p2l$ algorithm works very well and almost always converge to the correct answer **provided** that
- thare are **8** or more paired point-line, and
- the line orientation are not parallel.

The Python code is:

In [ ]:
import numpy as np
import numpy.matlib
import math

def p2l(X, Y, D, tol):
    """
    Computes the Procrustean point-line registration between X and Y+nD with 
    anisotropic Scaling,
        
    where X is a mxn matrix, m is typically 3
          Y is a mxn matrix denoting line origin, same dimension as X
          D is a mxn normalized matrix denoting line direction
          
          R is a mxm rotation matrix,
          A is a mxm diagonal scaling matrix, and
          t is a mx1 translation vector
          Q is a mxn fiducial on line that is closest to X after registration
          fre is the fiducial localization error
          
    based on the Majorization Principle
    """
    [m,n] = X.shape
    err = np.Infinity
    E_old = 1000000 * np.ones((m,n))
    e = np.ones((1,n))
    # intialization
    Q = Y
    # normalize the line orientation just in case
    Dir = D/np.linalg.norm(D, ord=2,axis=0,keepdims=True)
    while err > tol:
        [R, t, A] = AOPA_Major(X, Q, tol)
        E  = Q-np.matmul(R,np.matmul(A,X))-np.matmul(t,e)
        # project point to line
        Q = Y+Dir*np.matlib.repmat(np.einsum('ij,ij->j',np.matmul(R,np.matmul(A,X))+np.matmul(t,e)-Y,Dir),m,1)       
        err = np.linalg.norm(E-E_old)
        E_old = E
    E = Q - np.matmul(R, np.matmul(A,X)) - np.matmul(t,e)
    
    # calculate fiducial registration error
    fre = np.sum(np.linalg.norm(E,ord=2,axis=0,keepdims=True))/X.shape[1]
    return[R,t,A,Q,fre]